![example](images/director_shot.jpeg)

# Project Microsoft

**Author:** Rebecca Chu
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

### Rebecca's Preliminary Thoughts

Takeaway points of the business problem:
* Microsoft is looking to create a new movie studio and will be creating original video content
* Focus is on the types of films doing best
    * Genre, time of release, budget
* What are actionable insights regarding what type of films to create
    * Is there a specific genre we should be focusing on that guarantees commercial success?
    * Does time of release have an impact on success? Perhaps movies released in the summer are more popular as people are going out more?
    * Budget should have a relationship, the higher the budget, the higher the quality and star appeal of the movie. 

In [2]:
# import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [35]:
df1 = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
df1.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [36]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [5]:
# dropped the 'studio' column as it is not relevant to our business problem
df1_domestic = df1.drop(['studio', 'domestic_gross'], axis=1)
df1_domestic.head()

,title,foreign_gross,year
0,Toy Story 3,652000000,2010
1,Alice in Wonderland (2010),691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,664300000,2010
3,Inception,535700000,2010
4,Shrek Forever After,513900000,2010


In [8]:
df1_worldwide = df1[df1['foreign_gross'].notna()]
df1_worldwide = df1_worldwide[df1_worldwide['domestic_gross'].notna()]
df1_worldwide.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2009 entries, 0 to 3353
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2009 non-null   object 
 1   studio          2007 non-null   object 
 2   domestic_gross  2009 non-null   float64
 3   foreign_gross   2009 non-null   object 
 4   year            2009 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 94.2+ KB


In [13]:
df1_worldwide['foreign_gross'] = df1_worldwide['foreign_gross'].map(lambda x: x.strip(','))

In [17]:
pd.to_numeric(df1_worldwide['foreign_gross'])

ValueError: Unable to parse string "1,131.6" at position 1277

In [27]:
df1_worldwide['foreign_gross'][1276:1280]

1835      27000
1872    1,131.6
1873    1,019.4
1874    1,163.0
Name: foreign_gross, dtype: object

In [15]:
df1_worldwide['foreign_gross'].astype('float64')

ValueError: could not convert string to float: '1,131.6'

In [13]:
df3 = pd.read_csv('./zippedData/rt.movie_info.tsv.gz', sep='\t')
df3.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [17]:
# dropping currency as there is only one unique currency value
df3.currency.value_counts()

currency
$    340
Name: count, dtype: int64

In [19]:
df3.drop(['writer', 'dvd_date', 'currency', 'studio'], axis=1)

,id,synopsis,rating,genre,director,theater_date,box_office,runtime
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,"Oct 9, 1971",NaN,104 minutes
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,"Aug 17, 2012","600,000",108 minutes
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,"Sep 13, 1996",NaN,116 minutes
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,"Dec 9, 1994",NaN,128 minutes
4,7,NaN,NR,Drama|Romance,Rodney Bennett,NaN,NaN,200 minutes
...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,"Aug 18, 2006","33,886,034",106 minutes
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,"Jul 23, 1993",NaN,88 minutes
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,"Jan 1, 1962",NaN,111 minutes
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,"Apr 1, 1993",NaN,101 minutes


In [16]:
df3.info()
df3.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [21]:
df4 = pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep='\t', encoding='latin-1')

In [20]:
df4.top_critic.value_counts()

top_critic
0    41336
1    13096
Name: count, dtype: int64

In [21]:
df4.isna().sum()

id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64

In [71]:
df4.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [23]:
# drop publisher, date, and critic 
df5.drop(['critic', 'publisher', 'date'], axis=1)

,id,review,rating,fresh,top_critic
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,0
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,0
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,0
3,3,Continuing along a line introduced in last yea...,NaN,fresh,0
4,3,... a perverse twist on neorealism...,NaN,fresh,0
...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,1
54428,2000,NaN,1/5,rotten,0
54429,2000,NaN,2/5,rotten,0
54430,2000,NaN,2.5/5,rotten,0


In [91]:
df5 = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
df5.head() 

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [92]:
df5['genre_ids'] = df5.genre_ids.map(lambda x: x.strip('[ ]'))
df5.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"12, 14, 10751",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"14, 12, 16, 10751",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"12, 28, 878",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"16, 35, 10751",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"28, 878, 12",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [93]:
# split string based on ', ' (space is important)
df5['genre_ids'] = df5.genre_ids.map(lambda x: x.split(', '))

In [114]:
df5.vote_count.max()

22186

In [95]:
df5.original_language.value_counts()

original_language
en    23291
fr      507
es      455
ru      298
ja      265
      ...  
bo        1
si        1
sl        1
hz        1
dz        1
Name: count, Length: 76, dtype: int64

In [96]:
df6 = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')
df6.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [97]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [102]:
# converting release_date into datetime object
df6['release_date'] = pd.to_datetime(df6['release_date'])
df6.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [109]:
# creating a new column with release_month
df6['release_month'] = pd.DatetimeIndex(df6['release_date']).month
df6.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_month
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",12
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",5
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",6
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",5
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",12


## IMDb

Set index to 'tconst'
Dropping original_title column to go with the primary_title (more popular) also more missing values
Remove primary_title NaN 1 

Create new dataframe with runtime_minutes but otherwise drop as not related  
Split genre across columns

In [3]:
df7 = pd.read_csv('./zippedData/imdb.title.basics.csv.gz')
df7.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [4]:
df7 = df7.set_index('tconst')

In [5]:
df7 = df7.drop('original_title', axis=1)

In [6]:
df7 = df7[df7['primary_title'].notna()]

In [7]:
df7.genres.fillna('Unknown', inplace=True)

In [8]:
df7.isna().sum()

primary_title          0
start_year             0
runtime_minutes    31738
genres                 0
dtype: int64

In [10]:
number_of_genres_per_movie = []
for i in range(len(df7.genres)): 
    number_of_genres_per_movie.append(len(df7.genres[i]))

print(f"Maximum number of genres for a movie: {max(number_of_genres_per_movie)}")

Maximum number of genres for a movie: 3


In [9]:
df7['genres'] = df7.genres.map(lambda x: x.split(','))

In [26]:
print(f"Number of movies with 3 genres attributed: {number_of_genres_per_movie.count(3)}")
print(f"Number of movies with 2 genres attributed: {number_of_genres_per_movie.count(2)}")
print(f"Number of movies with 1 genres attributed: {number_of_genres_per_movie.count(1)}")

Number of movies with 3 genres attributed: 29435
Number of movies with 2 genres attributed: 29942
Number of movies with 1 genres attributed: 86766


In [13]:
df7['genre1'] = df7.genres.map(lambda x: x[0])

In [14]:
df7['genre2'] = df7.genres.map(lambda x: x[1] if len(x) > 1 else 'Not applicable')

In [34]:
# create a set of genres

list_of_genres = []
for i in range(len(df7.genres)):
    for i_g in range(len(df7.genres[i])):
        list_of_genres.append(df7.genres[i][i_g])

set(list_of_genres)

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'Unknown',
 'War',
 'Western'}

In [15]:
df7['genre3'] = df7.genres.map(lambda x: x[2] if len(x) > 2 else 'Not applicable')

In [16]:
df7

,primary_title,start_year,runtime_minutes,genres,genre1,genre2,genre3
tconst,,,,,,,
tt0063540,Sunghursh,2013,175.0,"[Action, Crime, Drama]",Action,Crime,Drama
tt0066787,One Day Before the Rainy Season,2019,114.0,"[Biography, Drama]",Biography,Drama,Not applicable
tt0069049,The Other Side of the Wind,2018,122.0,[Drama],Drama,Not applicable,Not applicable
tt0069204,Sabse Bada Sukh,2018,NaN,"[Comedy, Drama]",Comedy,Drama,Not applicable
tt0100275,The Wandering Soap Opera,2017,80.0,"[Comedy, Drama, Fantasy]",Comedy,Drama,Fantasy
...,...,...,...,...,...,...,...
tt9916538,Kuambil Lagi Hatiku,2019,123.0,[Drama],Drama,Not applicable,Not applicable
tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,[Documentary],Documentary,Not applicable,Not applicable
tt9916706,Dankyavar Danka,2013,NaN,[Comedy],Comedy,Not applicable,Not applicable


In [145]:
df7_genres = df7.drop('runtime_minutes', axis=1)

In [146]:
df7_genres

,primary_title,start_year,genres,genre1,genre2,genre3
tconst,,,,,,
tt0063540,Sunghursh,2013,"[Action, Crime, Drama]",Action,Crime,Drama
tt0066787,One Day Before the Rainy Season,2019,"[Biography, Drama]",Biography,Drama,Not applicable
tt0069049,The Other Side of the Wind,2018,[Drama],Drama,Not applicable,Not applicable
tt0069204,Sabse Bada Sukh,2018,"[Comedy, Drama]",Comedy,Drama,Not applicable
tt0100275,The Wandering Soap Opera,2017,"[Comedy, Drama, Fantasy]",Comedy,Drama,Fantasy
...,...,...,...,...,...,...
tt9916538,Kuambil Lagi Hatiku,2019,[Drama],Drama,Not applicable,Not applicable
tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,[Documentary],Documentary,Not applicable,Not applicable
tt9916706,Dankyavar Danka,2013,[Comedy],Comedy,Not applicable,Not applicable


In [35]:
df8.isna().sum()

averagerating    0
numvotes         0
dtype: int64

In [147]:
df8 = pd.read_csv('./zippedData/imdb.title.ratings.csv.gz')
df8.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [148]:
df8 = df8.set_index('tconst')

In [35]:
df8.isna().sum()

averagerating    0
numvotes         0
dtype: int64

In [152]:
df_imdb_genres = df7_genres.join(df8)
df_imdb_genres.head()

,primary_title,start_year,genres,genre1,genre2,genre3,averagerating,numvotes
tconst,,,,,,,,
tt0063540,Sunghursh,2013,"[Action, Crime, Drama]",Action,Crime,Drama,7.0,77.0
tt0066787,One Day Before the Rainy Season,2019,"[Biography, Drama]",Biography,Drama,Not applicable,7.2,43.0
tt0069049,The Other Side of the Wind,2018,[Drama],Drama,Not applicable,Not applicable,6.9,4517.0
tt0069204,Sabse Bada Sukh,2018,"[Comedy, Drama]",Comedy,Drama,Not applicable,6.1,13.0
tt0100275,The Wandering Soap Opera,2017,"[Comedy, Drama, Fantasy]",Comedy,Drama,Fantasy,6.5,119.0


## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [6]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***